In [1]:
import os
import json
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv, find_dotenv

_:bool = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] 
FMP_API_KEY = os.environ["FMP_API_KEY"] 

client: OpenAI = OpenAI()

# Define financial statement functions
def get_income_statement(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker, period, limit):
    # Code to fetch and return cash flow statement
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_cash_flow_statement(ticker, period, limit):
    # Code to fetch and return cash flow statement
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_key_metrics(ticker, period, limit):
    # Code to fetch and return cash flow statement
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_ratios(ticker, period, limit):
    # Code to fetch and return cash flow statement
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_growth(ticker, period, limit):
    # Code to fetch and return cash flow statement
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_cash_flow_statement,
    "get_financial_growth": get_financial_ratios
}

In [6]:
from openai.types.beta import Assistant

# def run_assistant(user_message: str):
assistant = client.beta.assistants.create(
    name="Financial Assistant",
    model="gpt-3.5-turbo-1106",
    instructions="Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends, etc, etc.",
    tools=[
        # {"type": "code_interpreter"},
        {
            "type": "function",
            "function": {
                "name": "get_income_statement",
                "description": "Get the income statement of the company",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                        "period": {"type": "string"},
                        "limit": {"type": "integer"}
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_balance_sheet",
                "description": "Get the balance sheet of the company",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                        "period": {"type": "string"},
                        "limit": {"type": "integer"}
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_cash_flow_statement",
                "description": "Get the cash flow statement of the company",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                        "period": {"type": "string"},
                        "limit": {"type": "integer"}
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_key_metrics",
                "description": "Get the company key metrics",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                        "period": {"type": "string"},
                        "limit": {"type": "integer"}
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_financial_ratios",
                "description": "Get the financial ratios of the company",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                        "period": {"type": "string"},
                        "limit": {"type": "integer"}
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_financial_growth",
                "description": "Get the company financial Growth",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                        "period": {"type": "string"},
                        "limit": {"type": "integer"}
                    },
                    "required": ["ticker"],
                },
            },
        }
    ]
)

thread = client.beta.threads.create()

messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you compare the financial health of Microsoft and Apple over the last four years, focusing on their balance sheets and key financial ratios?"
)
    

In [7]:
run = client.beta.threads.runs.create(
    assistant_id=assistant.id,
    thread_id=thread.id,
)

In [8]:
def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [9]:
while True:
    """
    This loop retrieves information about a run and performs actions based on its status.
    It checks if the run requires action, completes the run, handles failed runs, and waits for runs in progress or queued.
    """
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Add run steps retrieval here
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    print("Run Steps:", run_steps)

    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                response = function_to_call(**function_args)
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": response,
                })

        # Submit tool outputs and update the run
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )

    elif run.status == "completed":
        # List the messages to get the response
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
        print("Run failed.")
        break

    elif run.status in ["in_progress", "queued"]:
        print(f"Run is {run.status}. Waiting...")
        time.sleep(5)  # Wait for 5 seconds before checking again

    else:
        print(f"Unexpected status: {run.status}")
        break


Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_HMkmJkcKLb2rFJ5qGtIRyKQf', assistant_id='asst_tG0r2WXRrrPXQlLWIJUdwMeb', cancelled_at=None, completed_at=None, created_at=1707849337, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_ASKtqpCX9oh0dEei5LnL7BPB', status='in_progress', step_details=ToolCallsStepDetails(tool_calls=[FunctionToolCall(id='call_gU8ka4NFOr5PXh8vw8upnIwe', function=Function(arguments='{"ticker": "MSFT", "period": "quarterly", "limit": 16}', name='get_balance_sheet', output=None), type='function'), FunctionToolCall(id='call_BCW7a3p8fS6XPL89pUEAALkw', function=Function(arguments='{"ticker": "AAPL", "period": "quarterly", "limit": 16}', name='get_balance_sheet', output=None), type='function'), FunctionToolCall(id='call_3zbp1328HinRwccdwOLj4mTz', function=Function(arguments='{"ticker": "MSFT", "period": "annual", "limit": 4}', name='get_financial_ratios', output=None), type='function'), FunctionToolCall(id=